### Converting JSON to CSV file ##

In [63]:

import json
import csv
from datetime import datetime

# Function to parse and standardize dates
def parse_date(date_str):
    date_formats = [
        "%Y-%m-%d",        # 2023-01-15
        "%d/%m/%Y",        # 02/10/2023
        "%Y/%m/%d",        # 2023/05/18
        "%B %d, %Y",       # April 22, 2023
        "%m/%d/%Y",        # 08/05/2023
        "%m/%d/%Y",        # 4/22/2023
    ]
    for fmt in date_formats:
        try:
            date_obj = datetime.strptime(date_str, fmt)
            return date_obj.strftime("%d/%m/%Y")
        except ValueError:
            pass
    return date_str  # Return the original string if no format matches

# Function to convert JSON to CSV with results organized into two columns
def json_to_csv(json_file, csv_file):
    # Read the JSON data
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Prepare the output data structure
    output_data = []

    for record in data:
        experiment_id = record.get("experiment_id", "")
        description = record.get("description", "")
        metadata_date = parse_date(record.get("metadata", {}).get("date", ""))
        metadata_researcher = record.get("metadata", {}).get("researcher", "")
        metadata_location = record.get("metadata", {}).get("location", "")
        
        # Extract control and test values
        control_value = record.get("results", {}).get("control", "")
        test_value = record.get("results", {}).get("test", "")
        
        # Process other results, excluding control and test
        for result_type, result_value in record.get("results", {}).items():
            if result_type not in ["control", "test"]:
                output_data.append({
                    "experiment_id": experiment_id,
                    "description": description,
                    "result_type": result_type,
                    "result_value": result_value,
                    "control_result": control_value,
                    "test_result": test_value,
                    "metadata_date": metadata_date,
                    "metadata_researcher": metadata_researcher,
                    "metadata_location": metadata_location,
                })

    # Define the order of headers
    headers = [
        "experiment_id",
        "description",
        "control_result",
        "test_result",
        "result_type",
        "result_value",
        "metadata_date",
        "metadata_researcher",
        "metadata_location"
    ]

    # Write CSV data
    with open(csv_file, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(output_data)

# Example usage
json_file = 'experiments.json'  # Replace with JSON file path
csv_file = 'json_experiments.csv'  # Replace with desired CSV file path
json_to_csv(json_file, csv_file)

print(f"JSON data from {json_file} has been successfully written to {csv_file}")


JSON data from experiments.json has been successfully written to json_experiments.csv


### Converting XML to CSV file ##

In [64]:
import xml.etree.ElementTree as ET
import csv
from datetime import datetime

# Function to parse and standardize dates
def parse_date(date_str):
    date_formats = [
        "%Y-%m-%d",        # 2023-01-15
        "%d/%m/%Y",        # 02/10/2023
        "%Y/%m/%d",        # 2023/05/18
        "%B %d, %Y",       # April 22, 2023
        "%m/%d/%Y",        # 08/05/2023
        "%m/%d/%Y",        # 4/22/2023
    ]
    for fmt in date_formats:
        try:
            date_obj = datetime.strptime(date_str, fmt)
            return date_obj.strftime("%d/%m/%Y")
        except ValueError:
            pass
    return date_str  # Return the original string if no format matches

# Function to convert XML to CSV with results organized into two columns
def xml_to_csv(xml_file, csv_file):
    # Parse the XML data
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Prepare the output data structure
    output_data = []

    # Iterate over each experiment in the XML
    for experiment in root.findall('experiment'):
        experiment_id = experiment.find('experiment_id').text if experiment.find('experiment_id') is not None else ""
        description = experiment.find('description').text if experiment.find('description') is not None else ""
        
        metadata = experiment.find('metadata')
        metadata_date = parse_date(metadata.find('date').text) if metadata is not None and metadata.find('date') is not None else ""
        metadata_researcher = metadata.find('researcher').text if metadata is not None and metadata.find('researcher') is not None else ""
        metadata_location = metadata.find('location').text if metadata is not None and metadata.find('location') is not None else ""

        results = experiment.find('results')
        control_value = results.find('control').text if results is not None and results.find('control') is not None else ""
        test_value = results.find('test').text if results is not None and results.find('test') is not None else ""
        
        # Process other results, excluding control and test
        if results is not None:
            for result in results:
                result_type = result.tag
                result_value = result.text
                
                if result_type not in ["control", "test"]:
                    output_data.append({
                        "experiment_id": experiment_id,
                        "description": description,
                        "result_type": result_type,
                        "result_value": result_value,
                        "control_result": control_value,
                        "test_result": test_value,
                        "metadata_date": metadata_date,
                        "metadata_researcher": metadata_researcher,
                        "metadata_location": metadata_location,
                    })

    # Define the order of headers
    headers = [
        "experiment_id",
        "description",
        "control_result",
        "test_result",
        "result_type",
        "result_value",
        "metadata_date",
        "metadata_researcher",
        "metadata_location"
    ]

    # Write CSV data
    with open(csv_file, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(output_data)

# Example usage
xml_file = 'experiments.xml'  # Replace with XML file path
csv_file = 'XML_experiments.csv'  # Replace with desired CSV file path
xml_to_csv(xml_file, csv_file)

print(f"XML data from {xml_file} has been successfully written to {csv_file}")


XML data from experiments.xml has been successfully written to XML_experiments.csv


### Merging two CSVs file in one CSV file ##

In [65]:
import csv

# Function to merge two CSV files and exclude duplicates
def merge_csv_files_exclude_duplicates(file1, file2, output_file, unique_columns=None, sort_by=None):
    # Read the first CSV file
    with open(file1, 'r') as f1:
        reader1 = csv.DictReader(f1)
        data1 = list(reader1)
    
    # Read the second CSV file
    with open(file2, 'r') as f2:
        reader2 = csv.DictReader(f2)
        data2 = list(reader2)

    # Combine the headers from both files
    all_headers = set(reader1.fieldnames).union(set(reader2.fieldnames))
    
    # Define preferred header order
    preferred_order = ["experiment_id", "description","control_result","test_result","result_type","result_value"]
    
    # Sort the rest of the headers alphabetically
    remaining_headers = sorted(h for h in all_headers if h not in preferred_order)
    
    # Combine preferred order with the remaining headers
    all_headers = preferred_order + remaining_headers

    # Create a set to track unique rows
    seen_rows = set()

    # Function to create a unique key for each row based on specified columns
    def row_to_tuple(row):
        if unique_columns is not None:
            return tuple(row[col] for col in unique_columns if col in row)
        return tuple(row.get(header) for header in all_headers)

    # Merge the data, excluding duplicates
    merged_data = []
    for row in data1 + data2:
        row_tuple = row_to_tuple(row)
        if row_tuple not in seen_rows:
            seen_rows.add(row_tuple)
            merged_data.append(row)

    # Sort the merged data if a sort column is specified
    if sort_by:
        merged_data.sort(key=lambda x: x[sort_by])

    # Write the merged and sorted data to the output CSV file
    with open(output_file, 'w', newline='') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=all_headers)
        writer.writeheader()
        writer.writerows(merged_data)

# Reading
csv_file1 = 'json_experiments.csv'  # the path to first CSV file
csv_file2 = 'XML_experiments.csv'  # the path to second CSV file
output_csv_file = 'experiments_output.csv'  # desired output CSV file path
sort_column = 'experiment_id'  # Replace with the column want to sort by
unique_columns = ['experiment_id']  # Replace with the columns want to consider for uniqueness

merge_csv_files_exclude_duplicates(csv_file1, csv_file2, output_csv_file, unique_columns=unique_columns, sort_by=sort_column)

print(f"CSV files {csv_file1} and {csv_file2} have been successfully merged into {output_csv_file} without duplicates and sorted by {sort_column}.")


CSV files json_experiments.csv and XML_experiments.csv have been successfully merged into experiments_output.csv without duplicates and sorted by experiment_id.


### Reading as dataframe ##

In [66]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('experiments_output.csv')
df

,experiment_id,description,control_result,test_result,result_type,result_value,metadata_date,metadata_location,metadata_researcher
0,E123,Growth rate of plants under different light co...,0.2 cm/day,0.5 cm/day,growth_index,2.5,15/01/2023,Lab A,Dr. Green
1,E124,Effect of temperature on enzyme activity,No activity,Incresed activity at 37°C,temperature_effectiveness,0.8,02/10/23,Lab B,Dr. Blue
2,E125,Study of microbial growth in different pH levels,Neutral growth,NaN,growth_rate,NaN,12/03/2023,Lab C,Dr. Red
3,E126,Comparison of fuel efficiency in hybrid vs. el...,20 mpg,50 mpg,efficiency_increase,30%,22/04/2023,Field A,Dr. Yellow
4,E127,Effect of various diets on weight loss,1 kg/week,2.5 kg/week,weight_loss_ratio,2.5,18/05/2023,Lab D,Dr. White
5,E128,Impact of water hardness on detergent effectiv...,50% stain removal,80% stain removal,effectiveness_gain,30,22/06/2023,Lab E,Dr. Black
6,E129,Analysis of chemical reaction rates at differe...,Slow reaction,Rapid reaction at 60°C,reaction_time,15 mins,15/07/2023,Lab F,Dr. Orange
7,E130,Testing different water purification methods,0.5 ppm contaminants,0.1 ppm contaminants with new method,purity_increase,80%,08/05/2023,Lab G,Dr. Purple
8,E131,Effects of varying light intensities on solar ...,80% efficiency,90% efficiency at high intensity,efficiency_gain,10%,20/09/2023,Field B,Dr. Gray
9,E132,Comparative study of different insulation mate...,Standard insulation,Improved thermal resistance with new material,thermal_resistance,High,10/10/2023,Lab H,Dr. Brown
